In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from scipy.linalg import toeplitz


#----------Read from the file, find the entropy


colnames=['a'] 
df = pd.read_csv("rand1.txt", names=colnames, header=None, dtype=str) #Read the .txt file using pandas dataframe (=pd.DF)
#print(df)

x = len(df.axes[0]) #Find the number of rows in the pd.DF, required for probability and entropy
#print(x)

counts = df.value_counts() #Count the repitition/occurance (entries in rows) of each of the possible bit strings
#print(counts)

df2 = pd.DataFrame(counts, columns=list('a')) #Update pd.DF column with the rows being unique

prob = counts/x #Probability of each possible bit string
#print(prob)
df2['b'] = pd.DataFrame(prob) #Update pd.DF with prob as a new column of each row

df2['c'] = -(df2['b'] * np.log2(df2['b'])) #Find the entropy of the row elements and store in a new column
df3 = df2.sort_index() #Sort the rows with increasing bit string values
r_df3=len(df3.axes[0]) #Find the number of rows of the pd.DF 
c_df3=len(df3.axes[1]) #Find the number of columns of the pd.DF
#print("rows=",r_df3,", cols=",c_df3)
#df3['0'] = df3['0'].apply('{:0>8}'.format)
#print(df2)
#print(df3)
total = (np.sum(df2['c'].values)) #Find the Shannon entropy (sum of individual entropies), required for hashing
#print(total)

#df2["b"].plot()
#df3["b"].plot()
#plt.show()


#----------Using the value of entropy, find the dimensionality of the required topelitz matrix and create one


#c_tmat=int(total*10)
c_tmat=int(total) #Find the floor of the entropy 
#r_tmat=int(input("Enter the string length from ADC : ")) #Input the length of the bits from ADC
len_rmat=df['a'].iloc[0]
rmat=len(len_rmat)
r_mat1=r_tmat*1

randomlist = []
for i in range(0,r_tmat):
    n = random.randint(0,1)
    randomlist.append(n)
#print(randomlist) #Create a list of pseudo-random numbers (=PRN) of length equal to that of row # of pd.DF, required for tmat

tmat_full=toeplitz(randomlist) #Create toeplitz matrix (=tmat) from the list of PRN
#print(tmat_full)
np.savetxt("toeplitz_mat_full.txt",tmat_full, delimiter=",", fmt='%s')

tmat=np.delete(tmat_full, np.s_[c_tmat:r_tmat], axis=1) #Reduce the size of tmat to the required quantity
print(tmat)
np.savetxt("toeplitz_mat.txt",tmat, delimiter=",", fmt='%s')

df_tmat = pd.DataFrame(tmat, index=None) #Create a DF of the reduced toeplitz matrix (=t.DF)
r_df_tmat=len(df_tmat.axes[0]) #Find the # of rows in the t.DF
c_df_tmat=len(df_tmat.axes[1]) #Find the # of columns in the t.DF
#print("rows=",r_df_tmat,", cols=",c_df_tmat)
#print(df_tmat)


#----------Multiply the 12-bit string list file with the toeplitz matrix to get reduced dimensionality matrix


result_list=[]

r_df=int(x/8)
#print(r_df)
list_r_df=list(range(r_df))
#print(list_r_df)

df_split = np.array_split(df, r_df)
#df_split_list=df_split.values.tolist()
#print(df_split)

for y in list_r_df:
    list_df_split=(df_split[y]['a']).to_list()
    list_df_splitn=list(list_df_split)
    for i in list_df_splitn:
        list_final=list(i)
        list_final2=np.array(list_final)
        list_final3=pd.to_numeric(list_final2)
        matrix_multiply=np.matmul(list_final3, tmat)
        binary_matrix=np.mod(matrix_multiply,2)
        #print(list_final3)
        #print(matrix_multiply)
        #print(binary_matrix)
        result_list.append(binary_matrix)

print(result_list)
np.savetxt("reduced_matrix.txt",result_list, delimiter="\t", fmt='%s')


#-----------------------------------
#join 10 lines in list and then perform matmul


row_merged = df.groupby(df.index//5).agg(list)
row_merged['b'] = row_merged['a'].apply(lambda x: ''.join(map(str, x)))
print(row_merged)


#-----------------------------------
#check for trimming first/last few bits in the 12 bit string


row_merged['c'] = row_merged['b'].str[5:]
np.savetxt("trimmed_matrix.txt",row_merged['b'], delimiter=" ", fmt='%s')
print(row_merged)


#-----------------------------------
#Find the min-entropy of reduced matrix




#-----------------------------------
#Find randomness using NIST suite of all the three quantities



KeyboardInterrupt

